# Pair Programming ETL III

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [33]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
import ast

In [10]:
dicc = {"usa": [39.7837304, -100.445882], "australia":[-24.7761086, 134.755], "south africa":[-28.8166236, 24.991639],"new zealand": [-41.5000831, 172.8344077],"papua new guinea": [-5.6816069, 144.2489081] }

In [55]:
df_attack = pd.read_csv("../datos/datos_limpiezaV.csv", index_col=0)

In [59]:
class extraer_limpiar:
    
    def __init__(self, lat, lon, pais):
        
        self.lat = lat
        self.lon = lon
        self.pais = pais         

    def extraer_datos(self):
        
        df = pd.DataFrame()
    
        url ="http://www.7timer.info/bin/api.pl?lon={self.lon}&lat={self.lat}&product=meteo&output=json"
    
        response = requests.get(url = url) 
        response.status_code
    
        response_df = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
        response_df["country"] = self.pais
    
        df = pd.concat([df, response_df], axis = 0)
        return df 
    
    def limpiar(self,df1):

        x = df1['rh_profile'].apply(pd.Series) 

        for i in range(len(x.columns)): 
            nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
            valores = list(x[i].apply(pd.Series)["rh"] )
     
            df1.insert(i, nombre, valores)

        a = df1['wind_profile'].apply(pd.Series)

        for i in range(len(a.columns)): 
            nombre = "winddirection_" + str(a[i].apply(pd.Series)["layer"][0])
            valores = list(a[i].apply(pd.Series)["direction"])

            df1.insert(i, nombre, valores)

        for i in range(len(a.columns)): 
            
            nombre = "wind_speed_" + str(a[i].apply(pd.Series)["layer"][0])
            valores = list(a[i].apply(pd.Series)["speed"])
            
            df1.insert(i, nombre, valores)
        
        return df1

    def unir(self,df_clim,df_ata):

        df_group = df_clim.groupby('country').mean()
        df_group.reset_index()
        df_final = pd.merge(df_group, df_ata, how="inner", right_on="country", left_on="country")

        return df_final


In [60]:
api = extraer_limpiar(39.7837304, -100.445882,"usa")
api

In [61]:
df_extraido = api.extraer_datos()
df_extraido.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 190, 'speed':...",26,-1,12,-9999,none,0,0,190,3,usa
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 210, 'speed':...",26,-4,12,-9999,none,0,0,200,3,usa


In [62]:
df_extraido2 = api.limpiar(df_extraido)


In [63]:
df_extraido2.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,2,2,2,2,2,2,3,4,4,...,26,-1,12,-9999,none,0,0,190,3,usa
1,3,2,2,2,2,2,3,3,4,3,...,26,-4,12,-9999,none,0,0,200,3,usa
2,3,2,2,2,2,2,3,3,3,3,...,26,-1,11,-9999,none,0,0,185,3,usa
3,2,2,2,2,2,3,3,3,3,3,...,26,-1,11,-9999,rain,1,0,190,3,usa
4,2,2,2,2,2,2,3,3,3,2,...,26,-4,11,-9999,rain,2,0,200,2,usa


In [65]:
df_final = api.unir(df_extraido2,df_attack)
df_final.head()

,country,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,...,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,usa,2.421875,2.1875,2.484375,3.046875,3.28125,3.328125,3.28125,3.203125,3.484375,...,57,White shark,25-Jun-2018,Jun,n,F,white_shark,57.0,57.0,F
1,usa,2.421875,2.1875,2.484375,3.046875,3.28125,3.328125,3.28125,3.203125,3.484375,...,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark,15.0,15.0,M
2,usa,2.421875,2.1875,2.484375,3.046875,3.28125,3.328125,3.28125,3.203125,3.484375,...,25,"Tiger shark, 12'",31-Mar-2018,Mar,n,M,tiger_shark,25.0,25.0,M
3,usa,2.421875,2.1875,2.484375,3.046875,3.28125,3.328125,3.28125,3.203125,3.484375,...,54,Tiger shark,31-Dec-2017,Dec,n,F,tiger_shark,54.0,54.0,F
4,usa,2.421875,2.1875,2.484375,3.046875,3.28125,3.328125,3.28125,3.203125,3.484375,...,25,"White shark, 4.6 m",24-Nov-2017,Nov,n,M,white_shark,25.0,25.0,M
